In [1]:
from env_two_step import Scenario, AMoD
from MPC import MPC
import os
from util import mat2str
import subprocess
from collections import defaultdict
from tqdm import trange
import numpy as np

device = 'linux'
if device == 'windows':
    CPLEXPATH = "C:/Program Files/ibm/ILOG/CPLEX_Studio1210/opl/bin/x64_win64/"
elif device == 'mac':
    CPLEXPATH = "/Applications/CPLEX_Studio1210/opl/bin/x86-64_osx/"
elif device == 'linux':
    CPLEXPATH = "/opt/ibm/ILOG/CPLEX_Studio128/opl/bin/x86-64_linux/"
    

### Scenario 1

In [2]:
# three levels
scenario = Scenario(sd=10,demand_input = {(1,6):2, (0,7):2, 'default':0.1}, fix_price=True) # uni-directional
env = AMoD(scenario)
mpc = MPC(env, CPLEXPATH)

In [3]:
test_episodes = 100
max_steps = 100
epochs = trange(test_episodes) # build tqdm iterator for loop visualization
np.random.seed(10)

test_rewards = []
test_revenue = []
test_served_demand = []
test_rebalancing_cost = []
test_operating_cost = []
action_list = []

for episode in epochs:
    try:
        obs = env.reset()
        episode_reward = 0
        episode_revenue = 0
        episode_served_demand = 0
        episode_rebalancing_cost = 0
        episode_operating_cost = 0
        episode_action_list = []
        obs = env.reset()
        done = False
        while(not done):
            res_path='noreb'
            obs, reward, done, info = env.pax_step(CPLEXPATH=CPLEXPATH,PATH=res_path)
            episode_reward += reward 
            rebAction = [0]*(len(env.region)**2)
            obs, reward, done, info = env.reb_step(rebAction)
            episode_reward += reward
            episode_served_demand += info['served_demand']
            episode_rebalancing_cost += info['rebalancing_cost']
            episode_operating_cost += info['operating_cost']
            episode_revenue += info['revenue']
        epochs.set_description(f"Episode {episode+1} | Reward: {episode_reward:.2f} | Revenue: {episode_revenue:.2f} | ServedDemand: {episode_served_demand:.2f} \
| Reb. Cost: {episode_rebalancing_cost} | Oper. Cost: {episode_operating_cost:.2f}")
        #Adding the total reward and reduced epsilon values
        test_rewards.append(episode_reward)
        test_revenue.append(episode_revenue)
        test_served_demand.append(episode_served_demand)
        test_rebalancing_cost.append(episode_rebalancing_cost)
        test_operating_cost.append(episode_operating_cost)
        action_list.append(episode_action_list)
        break
    except KeyboardInterrupt:
        break

Episode 1 | Reward: 1596.00 | Revenue: 1710.00 | ServedDemand: 230.00 | Reb. Cost: 0.0 | Oper. Cost: 114.00:   0%|          | 0/100 [00:01<?, ?it/s]